In [19]:
# imports
import os
import collections
import pickle
import pandas as pd
from datetime import datetime
import math

In [20]:
TEX_PATH = 'umphbase-v1.0.0'
if not os.path.exists(TEX_PATH):
    os.mkdir(TEX_PATH)

### Get Data

In [21]:
# This is the last pull from the old version of ATU
date_of_last_pull = '2020-12-21'
path = 'v1.0.0/atu_pull_%s/' % date_of_last_pull
shows = pd.read_pickle('%s/shows_%s.pickle' % (path, date_of_last_pull))
songs = pd.read_pickle('%s/songs_%s.pickle' % (path, date_of_last_pull))
live_songs = pd.read_pickle('%s/live_songs_%s.pickle' % (path, date_of_last_pull))

In [23]:
# setup
shows['date'] = pd.to_datetime(shows['date'], format='%m-%d-%Y')
shows = shows.sort_values('date')

### Create LaTeX Tables

In [5]:
# Helpful dictionaries

special_keys = ['$\\wedge$','*','H']
set_code = {'One Set' : 'S1',
            'Set 1' : 'S1',
            'Set 2' : 'S2',
            'Set 3' : 'S3',
            'Quarter 1': 'Q1',
            'Quarter 2' : 'Q2',
            'Quarter 3' : 'Q3',
            'Quarter 4' : 'Q4',
            'Overtime' : 'OT',
            'Encore' : 'E1',
            '2nd Encore' : 'E2',
            '3rd Encore' : 'E3'}
trans_to_text = {None: '',
                 ',' : ',',
                 '>' : '\\textgreater',
                 '->' : '\\textrightarrow'}

In [6]:
# Helper functions

def fix_table(text, table_type = 'supertabular'):
    if type(text) == str:
        text = text.replace('tabular', table_type)
        text = text.replace('\\toprule', '')
        text = text.replace('\\bottomrule', '')
        return text
    else:
        text[0] = text[0].replace('tabular', table_type)
        text[-1] = text[-1].replace('tabular', table_type)
        for i in [1,-2]:
            text[i] = ''
        if table_type == 'longtable':
            text[4:4] = '\endhead \n'
        return text

def edit_table(file_name, table_type = 'supertabular'):
    with open(file_name,"r") as file: 
        text=file.readlines() 
    text = fix_table(text, table_type)
    with open(file_name,"w") as file: 
        file.writelines(text)
        
def clean_text(text):
    if text is not None:
        return (text.replace('\R', '\\\\R')
                    .replace('\E', '\\\\E')
                    .replace('%', '\\%')
                    .replace('é', "\\'e")
                    .replace('&', "\\&")
                    .replace('#', "\\#")
                    .replace('$', "\\$")
                    .replace('>', '\\textgreater \\enspace ')
                    .replace('^', '$\\wedge$')
                    .replace('_', '\\char`_'))
    else:
        return None
    
def clean_file_name(text):
    return (text.replace(' ', '_')
                .replace('.','')
                .replace('&','')
                .replace('"','')
                .replace('#','')
                .replace('/',''))

def table_list(ls, cols):
    rows = math.ceil(len(ls) / cols)
    table = {}
    for i in range(rows):
        table[i] = []
    for i in range(len(ls)):
        table[i % rows].append(ls[i])
    table = list(table.values())
    return pd.DataFrame(table).fillna('')

def order_sets(sets):
    ordered_sets = (['One Set'] + 
                ['Set %d' % (i) for i in range(1,4)] + 
                ['Quarter %d' % (i) for i in range(1,5)] + 
                ['Overtime', 'Encore', '2nd Encore', '3rd Encore'])
    tmp = []
    for i in ordered_sets:
        if i in sets:
            tmp.append(i)
    return tmp

def song_tex(row, title='name', tag=True, superscript=True, y='y'):
    '''Generate tex text for a song.'''
    text = row[title]
    if type(text) != str:
        text = text.strftime('%m-%d-%' + y) 
    text = clean_text(text)
    truth = list(row[['stewart','stewart_with_lyrics','hof']])
    special = any(truth)
    ss = ''.join([special_keys[i] for i in range(3) if truth[i]])
    ss = '' if not superscript else ss
    t = '' if not tag or row['tag'] == -1 else '[%d]' % (row['tag'])
    if special:
        return '\\textbf{%s%s\\textsuperscript{%s}}' % (text, t, ss)
    else:
        return '%s%s\\textsuperscript{%s}' % (text, t, ss)
    
def show_tex(show_key, title='date', superscript=True, y='y'):
    '''Generate tex text for a show.'''
    row = shows.loc[show_key]
    text = row[title]
    if type(text) != str:
        text = text.strftime('%m-%d-%' + y) 
    text = clean_text(text)
    tmp = live_songs[live_songs.show == show_key].sum()
    tmp = list(tmp[['stewart', 'stewart_with_lyrics', 'hof']])
    truth = [bool(x) for x in tmp]
    special = any(truth)
    ss = ''.join([special_keys[i] for i in range(3) if truth[i]])
    ss = '' if not superscript else ss
    if special:
        return '\\textbf{%s\\textsuperscript{%s}}' % (text, ss)
    else:
        return '%s\\textsuperscript{%s}' % (text, ss)

In [7]:
# Song codes
# ----------
path = '%s/song_codes.tex' % TEX_PATH 
tmp = songs.reset_index().rename(columns={'primary_key' : 'code'})
tmp = tmp.iloc[tmp.code.str.lower().argsort()]
tmp.at[tmp['name'].str.len() > 20, 'name'] = (
    tmp[tmp['name'].str.len() > 20]['name'].apply(lambda x: x[:18] + '..'))
tmp.to_latex(buf=path,
             columns=['code', 'name'],
             index=False,
             column_format='ll',
             header=False)
edit_table(path)

In [8]:
# Setlists
# --------

path = '%s/setlists.tex' % TEX_PATH 
dir_path = '%s/setlists' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
tmp = shows.sort_values('date')
f = open(path, "w")
for index, row in list(tmp.iterrows())[:5]:
    
    # for header
    date = row['date'].strftime('%m-%d-%Y')
    title = clean_text(row['title'])    
    
    f.write('\\noindent\n\\begin{minipage}{\\textwidth}\n')
    f.write('\\noindent\\underline{\\textbf{%s\\quad %s}} \\newline\n\n' % (date, title))
    
    # populate setlist table
    setlist_table = []
    show_songs = live_songs[live_songs.show == index]
    sets = order_sets(list(show_songs['set'].drop_duplicates())) 
    for set_name in sets:
        setlist = ''
        set_songs = show_songs[show_songs.set == set_name]
        n = int(list(set_songs[set_songs.out_transition.isna()]['order'])[0])
        for i in range(1,n+1):
            song = set_songs[set_songs.order == i].iloc[0]
            song = song.append(songs.loc[song['song']])
            trans = trans_to_text[song['out_transition']]
            trans = ' ' + trans if trans != ',' else trans
            setlist += "%s%s \\enspace " % (song_tex(song), trans)
        setlist_table.append(['\\textbf{%s}: ' % (set_code[set_name]),setlist])
    
    # create setlist table
    setlist_path = '%s/%s.tex' % (dir_path, index)
    if len(sets) > 0:
        main = pd.DataFrame(setlist_table)
        main.to_latex(buf=setlist_path,
                      index=False,
                      column_format='p{0.03\\textwidth}p{0.92\\textwidth}',
                      header=False,
                      escape = False)
        edit_table(setlist_path)
    else:
        f_tmp = open(setlist_path, "w")
        f_tmp.write('Setlist unavailable.\n')
        f_tmp.close()
            
    # tags, notes, and support text
    tags = row['tags']
    tag_text = ''
    if len(tags) > 0:
        tag_text += '\\begin{enumerate}\n'
        for i in tags:
            tag_str = clean_text('[%d] %s' % (i,tags[i]))
            tag_text += '\\item[] %s \n' % (tag_str)
        tag_text += '\\end{enumerate}\\vspace{5pt}\n\n'
    
    notes = row['notes']
    notes_text = ''
    if notes is not None and len(notes) > 0:
        notes_text += '\\noindent\\textbf{\\quad Notes:}\\vspace{3pt}\n\n\\begin{enumerate} \n'
        for note in notes:
            notes_text += '\\item[] %s \n' % (clean_text(note))
        notes_text += '\\end{enumerate}\\vspace{5pt}\n\n'
    
    support_text = clean_text(row['support'])
    support_text = '' if support_text is None else support_text
    if support_text != '':
        support_text = '\\textbf{\\quad Support:} %s\n\n' % (support_text)
       
    f.write('\\input{setlists/%s}\\vspace{5pt}\n\n%s%s%s' % (index, tag_text, notes_text, support_text))
    f.write('\\vspace{5pt}\n\n\\end{minipage}\n\n')
    
f.close()

In [9]:
# Songs Played
# ------------

path = '%s/songs_played.tex' % TEX_PATH 
tmp = live_songs.merge(shows.reset_index().rename(columns={'primary_key' : 'show'}), on='show')
tmp = tmp.sort_values('date')       
tmp = tmp.groupby('song')['date'].apply(list).reset_index(name='dates')
tmp['first_played'] = tmp['dates'].apply(lambda x: x[0].strftime('%m-%d-%y'))
tmp['last_played'] = tmp['dates'].apply(lambda x: x[-1].strftime('%m-%d-%y'))
tmp = (tmp.merge(songs.reset_index().rename(columns={'primary_key' : 'song'}), on='song')
          .sort_values('name')[['name', 'song', 'artist', 'first_played', 'last_played']])
tmp['name'] = tmp['name'].apply(clean_text)
tmp['artist'] = tmp['artist'].apply(clean_text)
tmp.to_latex(buf=path,
             index=False,
             column_format="p{0.35\\textwidth}"
                           "p{0.05\\textwidth}"
                           "p{0.35\\textwidth}"
                           "p{0.07\\textwidth}"
                           "p{0.07\\textwidth}",
             header=  ['\\textbf{%s}' % (x) for x in ['Name', 'Code', 'Artist', 'First Played', 'Last Played']],
             escape = False) 
edit_table(path, table_type='longtable')

In [10]:
# Song Plays by Year
# ------------------

path = '%s/songs_played_by_year.tex' % TEX_PATH 
tmp = live_songs[live_songs.parent]
tmp = live_songs.merge(shows.reset_index().rename(columns={'primary_key' : 'show'}), on='show')
tmp['year'] = tmp['date'].apply(lambda x: x.year)
tmp = tmp.groupby(['song','year']).count()['show'].to_dict()

ct_by_year = {}
for song in songs.index:
    year_ct = {}
    for y in range(1998,datetime.now().year+1):
        try:
            ct = tmp[(song,y)]
        except KeyError:
            ct = 0
        year_ct[str(y)[2:]] = ct
    ct_by_year[song] = year_ct
tmp = pd.DataFrame(ct_by_year).transpose()

header = ['\\textbf{%s}' % (str(x)[2:]) for x in list(range(1998,datetime.now().year+1))]
for i in range(3,25,6):
    header[i:i] = ' '
    tmp.insert(i, 'blank%s' % (i), '')
    
tmp = (tmp.reset_index().rename(columns={'index':'song'})
                        .merge(live_songs[live_songs.parent].groupby('song').count()['show'], on='song'))
header[0:0] = ['\\textbf{Song}']
header.append('\\textbf{Total}')
    
tmp.to_latex(buf=path,
             index=False,
             header=header,
             column_format=(''.join(["p{%f\\textwidth}" % (d) for d in [0.07] + 
                                     [0.01 if x == ' ' else 0.02 for x in header[1:-1]]])+'r'),
             escape = False) 
edit_table(path, table_type='longtable')

In [11]:
# Song instances
# --------------

path = '%s/every_time_played.tex' % TEX_PATH 
dir_path = '%s/every_time_played' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f = open(path, "w")
tmp_full = live_songs.merge(shows.reset_index().rename(columns={'primary_key' : 'show'}), on='show')
tmp_full = tmp_full.merge(songs.reset_index().rename(columns={'primary_key' : 'song'}), on='song')
tmp_full = tmp_full.sort_values('date')
tmp_full['date'] = tmp_full['date'].apply(lambda x : x.strftime('%m-%d-%y'))
for song in list(songs.index)[:5]:
    # get name of song and set file name
    name = songs.loc[song]['name']
    file_name = song
    
    # get every instance of this song
    instances = []
    tmp = tmp_full[tmp_full.song == song]
    for index, row in tmp.iterrows():
        show = row['show']
        set_text = set_code[row['set']]
        order = row['order']
        
        # initialize stats of interest
        date = song_tex(row, title='date', tag=False, superscript=False, y='y')
        prev_song = ''
        in_transition = ''
        out_transition = trans_to_text[row['out_transition']]
        next_song = ''
        
        # if not last song in set
        if out_transition != '':
            next_song_row = (live_songs[(live_songs.show == show) & 
                                        (live_songs.set == row['set']) & 
                                        (live_songs.order == order+1)].iloc[0])
            next_song = song_tex(next_song_row, title='song', tag=False, superscript=False)
        else:    
            next_song = '*C%s*' % (set_text)
            
        # if not first in set
        if order != 1:
            prev_song_row = (live_songs[(live_songs.show == show) & 
                                        (live_songs.set == row['set']) & 
                                        (live_songs.order == order-1)].iloc[0])
            prev_song = song_tex(prev_song_row, title='song', tag=False, superscript=False)
            in_transition = trans_to_text[next_song_row['out_transition']]
        else:
            prev_song = '*O%s*' % (set_text)
            
        instances.append([date, prev_song, in_transition, out_transition, next_song])
        
    song_path = '%s/%s.tex' % (dir_path, file_name)
    f.write('\\SongInstances{%s}{every_time_played/%s}\n' % (clean_text(name), file_name))
    instances = pd.DataFrame(instances)
    instances.to_latex(buf=song_path,
                       index=False,
                       column_format="p{0.07\\textwidth}"
                                     "p{0.06\\textwidth}"
                                     "p{0.02\\textwidth}"
                                     "p{0.02\\textwidth}"
                                     "p{0.06\\textwidth}",
                       header=False,
                       escape = False) 
    edit_table(song_path)
f.close()

In [18]:
# Jimmy Stewart / Lyrics / HOF / VIP
# ----------------------------------

def song_subset_list(subset, name, year, f):
    tmp = subset.merge(songs.reset_index().rename(columns={'primary_key': 'song'}), on='song')
    tmp = tmp.sort_values(['date', 'order'])
    tmp['date'] = tmp['date'].apply(lambda x: x.strftime('%m-%d-%y'))
    tmp['text'] = [song_tex(row, title='name', tag=False) for index, row in tmp.iterrows()]
    tmp = tmp[['date', 'text']]
    
    if len(tmp) > 0:
        tmp.to_latex(buf='%s/%s/%d.tex' % (TEX_PATH, name, year),
                     index=False,
                     column_format='p{0.25\\columnwidth}p{0.75\\columnwidth}',
                     header=False,
                     escape=False)
        edit_table('%s/%s/%d.tex' % (TEX_PATH, name, year))
        f.write('\\SongListByYear{%s}{%s/%d}' % (year, name, year))
        
path = '%s/stewart.tex' % TEX_PATH 
dir_path = '%s/stewart' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f_stewart = open(path, "w")

path = '%s/stewart_with_lyrics.tex' % TEX_PATH 
dir_path = '%s/stewart_with_lyrics' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f_stewart_with_lyrics = open(path, "w")

path = '%s/hof.tex' % TEX_PATH 
dir_path = '%s/hof' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f_hof = open(path, "w")
tmp = live_songs.merge(shows.reset_index().rename(columns={'primary_key': 'show'}), on='show')
for year in range(1998,datetime.now().year+1):
    year_tmp = tmp[(tmp.date > datetime(year,1,1)) & (tmp.date < datetime(year+1,1,1))] 
    song_subset_list(year_tmp[year_tmp.stewart], 'stewart', year, f_stewart)
    song_subset_list(year_tmp[year_tmp.stewart_with_lyrics], 'stewart_with_lyrics', year, f_stewart_with_lyrics)
    song_subset_list(year_tmp[year_tmp.hof], 'hof', year, f_hof)
f_stewart.close()
f_stewart_with_lyrics.close()
f_hof.close()


def show_subset_list(subset, name):
    '''Create a list of a subset of shows called name.'''
    tmp = subset.reset_index().sort_values('date')
    tmp['date'] = tmp['primary_key'].apply(lambda x : show_tex(x))
    tmp['venue'] = tmp['venue'].apply(clean_text)
    tmp = tmp[['date', 'venue']]
    tmp.to_latex(buf='%s/%s.tex' % (TEX_PATH, name),
                 index=False,
                 column_format='ll',
                 header=False,
                 escape=False)
    edit_table('%s/%s.tex' % (TEX_PATH, name))
    
show_subset_list(shows[shows.vip == True], 'vip')

In [13]:
# State Aggregate
# ---------------

path = '%s/location.tex' % TEX_PATH 
tmp = shows.reset_index().rename(columns={'primary_key': 'show'})
tmp['state'] = tmp['state'].fillna(' ')
ct_tmp = tmp.groupby(['state', 'country'], as_index=False).count()[['state', 'country','show']]

tmp = live_songs.merge(tmp, on='show')
tmp = (tmp.groupby(['state','country'], as_index=False).sum()
          [['state', 'country', 'stewart','stewart_with_lyrics','hof']])
tmp = tmp.merge(ct_tmp, on=['state', 'country'])
tmp[['stewart', 'stewart_with_lyrics', 'hof']] = tmp[['stewart', 'stewart_with_lyrics', 'hof']].astype(int)
tmp = tmp[['state', 'country', 'show', 'stewart','stewart_with_lyrics', 'hof']]
tmp = tmp.sort_values(['country','state'])
tmp.to_latex(buf=path,
             escape=False,
             index=False,
             column_format="p{0.3\\columnwidth}"
                           "p{0.3\\columnwidth}"
                           "p{0.05\\columnwidth}"
                           "p{0.05\\columnwidth}"
                           "p{0.05\\columnwidth}"
                           "p{0.05\\columnwidth}",
             header=['\\textbf{State}', '\\textbf{Country}', '', '', '', '' ])
edit_table(path)

In [14]:
# Venue
# -----

def simplify_venue_name(venue): 
    # identify real venue name and remove excess
    if '"' in venue:
        venue = venue.split('" ')[-1]
    # standardize
    venue = venue.replace('&', 'and')
    venue = venue.replace('Amphitheatre', 'Amphitheater')
    venue = venue.replace('Music and Art', '')
    venue = venue.replace(' Music ', ' ')
    return venue

# get number of special occurances
tmp = live_songs.merge(shows.reset_index().rename(columns={'primary_key' : 'show'}), on='show')
tmp['venue'] = tmp['venue'].apply(simplify_venue_name)
special_tmp = (tmp.groupby(['venue', 'city'], as_index=False).sum()
                  [['venue','city','stewart', 'stewart_with_lyrics', 'hof']])

# get shows
tmp = shows.reset_index().rename(columns={'primary_key' : 'show'})
tmp = tmp.sort_values('date')
tmp['venue'] = tmp['venue'].apply(simplify_venue_name)
ct_tmp = tmp.groupby(['venue', 'city'], as_index=False)['show'].agg(lambda x: x.tolist())
ct_tmp['ct'] = ct_tmp['show'].apply(lambda x: len(x))
ct_tmp = ct_tmp.rename(columns={'show':'show_list'})                                     

# merge
tmp = special_tmp.merge(ct_tmp, on=['venue', 'city'], how='right').fillna(0)
tmp[['stewart','stewart_with_lyrics','hof']] = tmp[['stewart','stewart_with_lyrics','hof']].astype(int)
tmp['venue'] = tmp['venue'].astype(str) + ', ' + tmp['city'].astype(str)
tmp = tmp.sort_values('venue')


path = '%s/venues.tex' % TEX_PATH 
dir_path = '%s/venues' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f = open(path, "w")
small_venues = []
for index, row in tmp.iterrows():
    venue = clean_text(row['venue'])
    file_name = venue.replace(' ', '_').replace('.','').replace('/','').replace('"','')
    stats = list(row[['ct','stewart','stewart_with_lyrics','hof']])
    stats_string = '\\qquad'.join([str(x) for x in stats])
    show_list = row['show_list']
    date_list = [show_tex(x, superscript=False) for x in show_list]   
    if len(date_list) >= 3:
        venue_path = '%s/%s.tex' % (dir_path, file_name)
        dates = table_list(date_list,3)
        f.write('\\VenueSummary{%s}{%s}{venues/%s}\n' % (venue, stats_string, file_name))
        dates.to_latex(buf=venue_path,
                       index=False,
                       column_format="p{0.33\\columnwidth}"
                                     "p{0.33\\columnwidth}"
                                     "p{0.33\\columnwidth}",
                       header=False,
                       escape=False)
        edit_table(venue_path)
    else:
        small_venues.append([venue] + [', '.join(date_list)] + stats)
    
path = '%s/venues_sm.tex' % TEX_PATH 
pd.DataFrame(small_venues).to_latex(buf=path,
                                   index=False,
                                    column_format="p{0.5\\textwidth}"
                                                  "p{0.25\\textwidth}"
                                                  "p{0.03\\textwidth}"
                                                  "p{0.03\\textwidth}"
                                                  "p{0.03\\textwidth}"
                                                  "p{0.03\\textwidth}",
                                    header=['\\textbf{Venue}', '\\textbf{Shows}', '', '', '', '' ],
                                    escape=False)
edit_table(path, table_type='longtable')
f.close()

In [15]:
# Support
# -------

path = '%s/support.tex' % TEX_PATH 
dir_path = '%s/support' % TEX_PATH
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
f = open(path, "w")
support_acts = list(shows['support'].drop_duplicates())
support_acts.remove(None)
support_acts.sort()
for support in support_acts:
    file_name = support.replace(' ', '_').replace('.','')
    tmp = shows[shows.support == support][['date', 'venue']].sort_values('date')
    tmp['date'] = tmp['date'].apply(lambda x : x.strftime('%m-%d-%y'))
    f.write('\\SupportSummary{%s}{support/%s}\n' % (clean_text(support), file_name))
    support_path = '%s/%s.tex' % (dir_path, file_name)
    tmp.to_latex(buf=support_path,
                 index=False,
                 column_format='p{0.25\\columnwidth}p{0.75\\columnwidth}',
                 header=False)
    edit_table(support_path)
f.close()

In [16]:
# Reviews
# -------

path = '%s/reviews.tex' % TEX_PATH 
f = open(path, "w")
tmp = shows[~shows.reviews.isna()]
for index, row in tmp.iterrows():
    date = row['date'].strftime('%m-%d-%y')
    show = clean_text(row['title'])
    for review in row['reviews']:
        author = clean_text(review['reviewer'])
        text = clean_text(review['text'].replace('\n', ' ')
                                        .replace('person found this helpful', '')
                                        .replace('people found this helpful', '')
                                        .rstrip('1234 '))
        if len(text) > 256 and 'http://' not in text:
            f.write('\\Review{%s}{%s}{%s}{%s}\n' % (date, show, author, text))
f.close()            